In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from train_predict_utils import *

In [ ]:
class LSTMModel1(nn.Module): 
    # basic one with two linear layers and final output with sigmoid
    def __init__(self, input_size, hidden_size=1024, num_layers=1):
        super().__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size,
                            batch_first=True, dropout=0, num_layers=num_layers)
        self.fc1 = nn.Linear(hidden_size, 4000)
        self.fc2 = nn.Linear(4000, 4000)
        self.fc3 = nn.Linear(4000, input_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        # hidden state
        h_0 = torch.zeros(self.num_layers * 1, x.size(0),
                          self.hidden_size).to(x.device)  # Modified line
        # cell state
        c_0 = torch.zeros(self.num_layers * 1, x.size(0),
                          self.hidden_size).to(x.device)  # Modified line

        out, (hn, cn) = self.lstm(x, (h_0, c_0))
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        out = self.fc3(out)
        out = torch.sigmoid(out)

        return out


In [ ]:
data_dir = "/sdf/group/lcls/ds/scratch/s2e_scratch/Data/SFG_reIm/version1"
train_dataset = CustomSequence(data_dir, range(80), file_batch_size=1, model_batch_size=512)
val_dataset = CustomSequence(data_dir, range(80, 81), file_batch_size=1, model_batch_size=512)


In [ ]:
model = LSTMModel1(input_size=8264)

In [ ]:
train(model, train_dataset, num_epochs=10, val_dataset=val_dataset, use_gpu=True, data_parallel=True, out_dir=".", model_name="model", verbose=1, save_checkpoints=True, custom_loss=None)

In [ ]:
test_dataset = CustomSequence(data_dir, range(90, 92), file_batch_size=1, model_batch_size=512, test_mode=True)

In [ ]:
predict(model, model_param_path="model_epoch_2.pth", test_dataset=test_dataset, use_gpu=True, data_parallel=False, output_dir=".", output_name="all_preds.npy", verbose=1)